### 1. Set environmental variables

In [1]:
from env_config import env_config

config = env_config()

2025-06-11 09:41:34,565 - INFO - Found a local .env file at /Users/drew_wilkins/Drews_Files/Drew/Python/Localcode/.env
2025-06-11 09:41:34,565 - INFO - Running in context: CLI
2025-06-11 09:41:34,565 - INFO - Force user authentication: False
2025-06-11 09:41:34,566 - INFO - Set balance of env values from.env file at /Users/drew_wilkins/Drews_Files/Drew/Python/Localcode/.env


OPTIONAL: Review environmental variables
WARNING: DO NOT COMMIT WITH THIS CELL OUTPUT SHOWING

In [ ]:
from pprint import pprint

pprint(config)

### 2. Init clients

In [2]:
from gcp_utils import get_gcp_credentials, init_sheets_client, init_drive_client
from qdrant_utils import init_qdrant_client


creds = get_gcp_credentials()
sheets_client = init_sheets_client(creds)
drive_client = init_drive_client(creds)
qdrant_client = init_qdrant_client("cloud")

2025-06-11 09:41:37,750 - INFO - Found a local .env file at /Users/drew_wilkins/Drews_Files/Drew/Python/Localcode/.env
2025-06-11 09:41:37,750 - INFO - Running in context: CLI
2025-06-11 09:41:37,750 - INFO - Force user authentication: False
2025-06-11 09:41:37,751 - INFO - Set balance of env values from.env file at /Users/drew_wilkins/Drews_Files/Drew/Python/Localcode/.env
2025-06-11 09:41:38,063 - INFO - Found a local .env file at /Users/drew_wilkins/Drews_Files/Drew/Python/Localcode/.env
2025-06-11 09:41:38,064 - INFO - Running in context: CLI
2025-06-11 09:41:38,064 - INFO - Force user authentication: False
2025-06-11 09:41:38,065 - INFO - Set balance of env values from.env file at /Users/drew_wilkins/Drews_Files/Drew/Python/Localcode/.env
2025-06-11 09:41:38,111 - INFO - file_cache is only supported with oauth2client<4.0.0
2025-06-11 09:41:38,113 - INFO - Initializing Qdrant client in 'cloud' mode...
2025-06-11 09:41:38,636 - INFO - HTTP Request: GET https://0c82e035-1105-40f2-a0b

### 3. Test main orcehstration functions

Validate rows in LIBRARY_UNIFIED

In [ ]:
from gcp_utils import fetch_sheet_as_df
from library_utils import validate_all_rows_format
from IPython.display import display, HTML


library_df = fetch_sheet_as_df(sheets_client, config["LIBRARY_UNIFIED"])
valid_df, invalid_df, log_df = validate_all_rows_format(library_df)

if not log_df.empty:
    print(f"\nValidation error detail:")
    display(HTML(log_df.to_html(notebook=True,
            max_cols=30, max_rows=10, escape=False)))

if not invalid_df.empty:
    print(f"\nInvalid rows detail:")
    display(HTML(invalid_df.to_html(notebook=True,
            max_cols=30, max_rows=10, escape=False)))

Promote tagged

In [ ]:
from promote_files import promote_files

promote_files(drive_client, sheets_client, qdrant_client)

Delete tagged


In [ ]:
from delete_tagged import delete_tagged


deleted_rows_df = delete_tagged(drive_client, sheets_client, qdrant_client)

cleanup orphans

In [ ]:
from cleanup_orphans import find_orphans
from IPython.display import display, HTML


orphan_rows, orphan_files, orphan_qdrant_records, log_df = find_orphans(
    drive_client, sheets_client, qdrant_client)

print(f"\nOrphan rows:")
if not orphan_rows.empty:
    display(HTML(orphan_rows.to_html(notebook=True,
            max_cols=30, max_rows=10, escape=False)))

print(f"\nOrphan files:")
if not orphan_files.empty:
    display(HTML(orphan_files.to_html(notebook=True,
            max_cols=30, max_rows=10, escape=False)))

print(f"\nOrphan qdrant records:")
if not orphan_qdrant_records.empty:
    display(HTML(orphan_qdrant_records.to_html(notebook=True,
            max_cols=30, max_rows=10, escape=False)))

### 4. Test helper functions

In [12]:
pdf_ids = [
    "f43bdf6d-07ac-5ce0-b206-db7eabb990ca",
    "5098752e-4166-55f2-9f46-227901cdfc9c",
    "1ff90293-9f4e-568a-a3ac-1a2e9a01838d",
    "b4b536dc-7acc-5ac4-a200-e92bfc3447a3",
    "712095dd-af64-5522-8d96-ce745c700fc2",
    "8c9e8a61-0c9c-5650-9020-ce6a07624716",
    "a23943b3-c50d-5a0e-8094-4782d5022e25",
    "08b3baae-3aa2-5ccd-8c80-8ab255a45156",
    "f7b993d6-1ce8-541b-81d6-0db6076062ca",
    "959f3f11-e812-5b59-b2f1-07f76a9025ef",
    "d10522f0-1909-55e2-b03c-6eef68f39bef",
    "41363a2d-0a9c-5e19-9b57-f85cb0e8f3fe",
    "12abc1d1-bec1-58b2-8763-7158f77e8698",
    "2d4143fd-7432-577e-8f5f-ac482185d292",
    "d354afd1-95ea-5489-9df7-3abdd322d73c",
    "09544186-4593-5579-b2af-09fb953b7e86",
    "4b7c9994-8f65-5a1d-8e8a-7d73f4b38de8",
    "21126abd-5489-5ffa-b566-d6f365d5a74b",
    "70e0849b-8f89-5242-9a40-c10b88173ac1",
    "2d6a042f-0c21-54b6-bd4f-588b8be73999",
    "75fddd31-e90f-5311-a38c-6d6f807d1fdc",
    "52d35e10-7df5-5ab8-b2a0-110c26c6d2b2",
    "3b845a10-cb3a-5014-96d8-360c8f1bf63f",
    "ba4883c2-9545-5d70-868a-867eea0a7678",
    "e214b5b8-9077-555d-b101-0b7eb5fef90a",
    "9edb0da8-7257-59d4-9b27-5bcede517047",
    "4527cfc9-f47d-5945-8e0d-9e98e1fce929",
    "c15f7a15-6f60-5064-870c-123a26d86bc6",
    "cc43c3f2-f44f-5df0-aab8-3f50359b0292",
    "9e8faabf-0fbd-5bcf-b7e1-00b859360f17",
    "bc8c6a12-932a-51ca-b40d-13fb1e6ab0a9",
    "c7143b60-c938-5329-9a3e-4d3411a2827b",
    "c820d16d-b010-5ca1-bd76-f877dc987c93",
    "23746b1e-b829-55fc-8777-8c85ec3a9f41",
    "7b5a850a-b5b2-50cf-bc87-1d5a0c92b322",
    "149e15a5-8f4f-53a6-a3dc-1becb28560d4",
    "d2c55494-6863-5186-a423-e83e32fad3b3",
    "042ba9b7-ee51-541f-9ec5-71ed26104824",
    "29d16974-c3a5-5c89-8997-262037815c93",
    "25e956d7-3dd1-5126-b9b9-2407e53e2121"
]

In [13]:
from IPython.display import display, HTML
from env_config import rag_config
from qdrant_utils import get_summaries_by_pdf_id


summaries_df = get_summaries_by_pdf_id(
    qdrant_client, rag_config("qdrant_collection_name"), pdf_ids)

if not summaries_df.empty:
    display(HTML(summaries_df.to_html(notebook=True,
            max_cols=30, max_rows=10, escape=False)))

2025-06-11 09:44:04,934 - INFO - HTTP Request: POST https://0c82e035-1105-40f2-a0bd-ecc44a016f15.us-east4-0.gcp.cloud.qdrant.io:6333/collections/ASK_vectorstore/points/scroll "HTTP/1.1 200 OK"


,pdf_id,title,pdf_file_name,page_count,record_count,point_ids
0,c7143b60-c938-5329-9a3e-4d3411a2827b,"COAST GUARD NAVIGATION STANDARDS MANUAL, COMDT...",None,93,80,"[000ee01d-2811-4500-b847-ac8486430938, 062b620..."
1,2d6a042f-0c21-54b6-bd4f-588b8be73999,"CH-3 TO COMDTINST M16000.9, MARINE SAFETY MANU...",None,452,452,"[000f377e-8aa9-4f4d-ae0d-725215542ed3, 0039348..."
2,70e0849b-8f89-5242-9a40-c10b88173ac1,"CH-2 TO MARINE SAFETY MANUAL VOLUME II, MATERI...",None,1343,1340,"[000f7189-21ec-4e40-a8bd-b44aee74c37c, 002b30c..."
3,bc8c6a12-932a-51ca-b40d-13fb1e6ab0a9,Navigation Rules & Regulations Handbook 85 FR ...,None,202,200,"[001e8dd0-16e1-4187-beda-83821d929b40, 003dfc0..."
4,c15f7a15-6f60-5064-870c-123a26d86bc6,"CH-15 TO MARINE SAFETY MANUAL, VOLUME I, ADMIN...",None,497,497,"[002e9c5f-e517-4d50-8deb-7d139b3759e1, 00c7cb5..."
...,...,...,...,...,...,...
35,a23943b3-c50d-5a0e-8094-4782d5022e25,"BSX Policy Letter AUX-PL-023(B)/23-03, UPDATE ...",None,2,2,"[2fc01041-7f71-4936-a877-d30356cd9eba, 8c297c3..."
36,09544186-4593-5579-b2af-09fb953b7e86,"CVC Policy Letter 18-03, UNINSPECTED PASSENGER...",None,7,7,"[3b5624ee-eb01-4fd5-a58b-6f54f313eeb5, 73e6c06..."
37,12abc1d1-bec1-58b2-8763-7158f77e8698,BSX Policy Letter AUX-PL-003(A)/19-03 AUXILIAR...,None,3,3,"[3da30c37-28f3-496d-aa19-0fc9e51ac019, 8bc1381..."
38,959f3f11-e812-5b59-b2f1-07f76a9025ef,"BSX Policy Letter AUX-PL-028(A)/24-03, REMOVAL...",None,4,4,"[4ece460e-5cea-4173-91ec-d10868af0fdb, 517dd26..."
